In [85]:
import cv2
from ultralytics import YOLO
import cvzone
import math

from sort import *

In [86]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [87]:
model = YOLO('./Yolo-Weights/yolov8n.pt')

In [88]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [89]:
mask = cv2.imread('./mask.png')

In [90]:
# creating a instance for tracking
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)

# we will make a line and count after that 
limits = [400,297,673,297]
totalcounts=0

In [91]:
while True:
    success, img = cap.read()
    #overlay mask on the image/webcam feed
    imageRegion = cv2.bitwise_and(img,mask)
    # results = model(img,stream=True)
    results = model(imageRegion,stream=True)
    
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                # cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                # cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=5) #using cvzone rectange
                currentArrays = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArrays))
            
    resultsTracker = tracker.update(detections)
    
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,255),5)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id = result
        # converting them into integers instead for tensor 
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h = x2-x1 , y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=2,colorR=(255,0,0))
        # cvzone.putTextRect(img,f' {int(id)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
        
        cx , cy = x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
        
        if limits[0] <cx< limits[2] and limits[1]-20<cy<limits[1]+20:
            totalcounts += 1
            cvzone.putTextRect(img,f' {int(totalcounts)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
        
    cvzone.putTextRect(img,f'Count : {totalcounts}',(90,90)) 
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imageRegion)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(0) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 4 cars, 162.5ms
Speed: 8.0ms preprocess, 162.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0.82
0.79
0.49
0.39
[        572         322         651         429         331]
[        573         364         626         429         330]
[        378         348         497         457         329]
[        456         225         510         270         328]



0: 384x640 4 cars, 100.0ms
Speed: 2.9ms preprocess, 100.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0.8
0.77
0.53
0.52
[     568.48      320.73      651.52      436.27         331]
[     572.26      371.69      627.74      438.31         330]
[     373.12      349.89      492.88      459.11         329]
[     454.74      228.22      508.26      273.78         328]



0: 384x640 2 cars, 152.8ms
Speed: 7.0ms preprocess, 152.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 80.9ms
Speed: 0.0ms preprocess, 80.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


...

0.85
0.77
0.75
0.73
0.45
[     557.65      206.61      606.72      247.91         345]
[     615.98      234.08      662.27      287.78         344]
[     569.63      304.16      642.47      385.27         343]
[     463.55       238.5      510.68      280.47         342]
0.85
0.75
0.74
0.63
0.61


Speed: 0.0ms preprocess, 90.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



[      502.9      200.29       542.9      223.14         347]
[     551.91      212.92      602.89      257.61         345]
[     610.97      238.17       663.4      299.62         344]
[     558.73      322.33      635.59      411.13         343]
[     457.03       243.3      505.89      287.43         342]


0: 384x640 6 cars, 85.6ms
Speed: 2.8ms preprocess, 85.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0.88
0.76
0.7
0.65
0.63
0.42
[     501.94      200.74      543.87      226.32         347]
[     545.62      220.86      598.49      268.11         345]
[      607.9      244.82      663.89      310.27         344]
[        545      344.38      628.36      441.72         343]
[     451.24      247.69      502.36      293.13         342]
